# Embedtree2
Notebook to compile the so far research of nvgtt.

In [1]:
import requests
import nltk
from bs4 import BeautifulSoup
import re
import pickle

ae


### Dict Storage class to be used to load and save dicts
It saves data in pickle format and works with lower case only to save memory

In [2]:
class DictStorage(dict):
    
    def __init__(self, filename):
        self.filename = filename + ".pickle"
        
        try:
            with open(self.filename, mode='r') as pickle_file:
                dict_list = pickle.load(pickle_file)
        except IOError:
            dict_list = []
            print("Failed to open " + self.filename + ". Created empty dict.")
        finally:
            dict.__init__(self, dict_list)
    
    def save(self):
        with open(self.filename, mode='w') as pickle_file:
            pickle.dump(self.items(), pickle_file)
            
    def __setitem__(self, key, value):
        super(DictStorage, self).__setitem__(key.lower(), value)

    def __getitem__(self, key):
        return super(DictStorage, self).__getitem__(key.lower())

In [9]:
def test_dictstorage():
    test1 = DictStorage("ae2")
    print(test1.items())
    print(test1.save())
    test1['LUCAS'] = 128
    print(test1.items())
    print(test1.save())
    print(test1['lucas'])
    print(test1['LUCAS'])
    
#test_dictstorage()

[('lucas', 128)]
None
[('lucas', 128)]
None
128
128


### Function to get wikipedia data

In [13]:
def get_page(page, lang="en"):
    """Function to retrieve a wikipedia page in html form, with its sections"""

    # https://en.wikipedia.org/w/api.php?action=parse&redirects&page=fluid_mechanics

    wikipediaApiUrl = "https://" + lang + ".wikipedia.org/w/api.php"

    pageParams = {
        'action': 'parse', 
        'redirects': True,
        'page': page,
        'format': 'json',
        'prop':'text|displaytitle'
    }

    pageData = requests.get(wikipediaApiUrl, pageParams).json()

    if not 'parse' in pageData:
        raise "Error while getting page " + page


    docHtml = BeautifulSoup(pageData['parse']['text']['*'], 'html.parser')

    #Split document by its sections
    docSections = __splitIntoSections__(docHtml)

    structPageData = {
        'title': pageData['parse']['title'],
        'pageid': pageData['parse']['pageid'],
        'full': docHtml,
        'sections': docSections
    }

    return structPageData


def __splitIntoSections__(htmlObj):
    """Function to split html document in sections (use h2 tags as divisors)"""

    #Init var to store sections
    sectionObjs = [[]]

    for tag in htmlObj.children:
        #Start new section in case the tag is h2
        if tag.name == 'h2':
            sectionObjs.append([])

        #If it is a valid tag (invalid tags has no 'name' property)
        if tag.name != None:
            sectionObjs[len(sectionObjs) - 1].append(tag)

    return sectionObjs
    

In [14]:
def get_page_and_parse(page):
    """Function to treat the data, remove unecessary things etc."""
    
    page_data = get_page(page)
    
    soup = page_data['full']
        
    #Clear table of contents if any
    for node in soup.findAll(id='toc'):
        node.decompose()

    #Clear top info table if any
    for node in soup.findAll(class_='ambox'):
        node.decompose()

    #Clear info box if any
    for node in soup.findAll(class_='infobox'):
        node.decompose()

    #Clear verticalbox if any
    for node in soup.findAll(class_='vertical-navbox'):
        node.decompose()
        
    return page_data

In [15]:
blocked_link_terms = set([
    "(disambiguation)", #Not interested in disambiguation pages
    ":" #Pages with colon are offen special pages. Not sure if there is articles with colon
])

def get_page_links(page_data):
    """Function to get links """
    #page_data = get_page_and_parse(page)
    
    links = list()
    
    for link in page_data['full'].findAll("a"):
        
        #If the a tag has no href attr, skip it
        if not link.has_attr("href"):
            continue
            
        #If the href does not starts with "/wiki/", skip it
        if link['href'].find("/wiki/") != 0:
            continue
            
        #Check if some blocked term is present in the href, if so, skip the link
        skip_link = False
        for term in blocked_link_terms:
            if link['href'].find(term) != -1:
                skip_link = True
                break
        if skip_link:
            continue
        
        #Get only the link portion
        #We MUST NOT use last index of / to get the path cause some titles like TCP/IP, have bar in the title
        #We should use the '/wiki/' string length
        linkHref = link['href'][6:]
        
        #Remove hashtag from url if any
        hashIndex = linkHref.find("#")
        if hashIndex != -1:
            linkHref = linkHref[:hashIndex]
            
        linkText = link.get_text()
            
        links.append((linkHref, linkText))
            
    return links#, n_valid_links

In [16]:
def get_page_links_score(links, text):
    """Function to cross a list of links with a text, setting scores."""
    
    links_score = dict()
    
    for link_href, link_texts in links.items():
        for l_text in link_texts:
            #GOT TO ENSURE l_text is not a stop word and is surrounded by non letters
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', text, re.IGNORECASE)
            links_score[link_href] = len(matches)
            
    return links_score
    

## Main()

In [97]:
wikisyn = DictStorage("wikisyn") #Storage for link synoms
page_redirects = DictStorage("page-redirects") #Storage lookup table of hrefs and pageids
#page_links_score = DictStorage("page-links-score") #Storage for page links scores
page_links = DictStorage("page-links") #Storage for page links
page_texts = DictStorage("page-texts") #Storage for page texts

Failed to open wikisyn.pickle. Created empty dict.
Failed to open page-links.pickle. Created empty dict.
Failed to open page-texts.pickle. Created empty dict.


In [ ]:
def get_page_data_from_db(page):
    page_lc = page.lower()
    
    #Check if the page is in the redirects table, if not, download it and register it
    if not page_redirects.has_key(page_lc):
        print("Page not found. Downloading and registering it...")
        page_data = get_page_and_parse(page)
        page_redirects[page_lc] = page_data['pageid']
        page_redirects[page_data['title'].lower()] = page_data['pageid']
        page_redirects.save()
        print("Done.")

In [94]:
target_page = "MQTT"

In [95]:
#Check if the page is in the redirects table, if not, download it and register it
if not page_redirects.has_key(target_page):
    print("Page not found. Downloading and registering it...")
    page_data = get_page_and_parse(target_page)
    page_redirects[target_page] = page_data['pageid']
    page_redirects.save()
    print("Done.")

In [96]:
page_data = get_page_and_parse(target_page)
print(page_data)

{'sections': [[<None></None>, <p><b>MQTT</b><sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> (<b>MQ Telemetry Transport</b> or <b>Message Queue Telemetry Transport</b>) is an ISO standard (ISO/IEC PRF 20922)<sup class="reference" id="cite_ref-ISO_2-0"><a href="#cite_note-ISO-2">[2]</a></sup> <a href="/wiki/Publish%E2%80%93subscribe_pattern" title="Publish\u2013subscribe pattern">publish-subscribe</a>-based "lightweight" messaging protocol for use on top of the <a class="mw-redirect" href="/wiki/TCP/IP" title="TCP/IP">TCP/IP protocol</a>. It is designed for connections with remote locations where a "small code footprint" is required or the network bandwidth is limited. The <a href="/wiki/Publish%E2%80%93subscribe_pattern" title="Publish\u2013subscribe pattern">publish-subscribe messaging pattern</a> requires a <a href="/wiki/Message_broker" title="Message broker">message broker</a>. The <a class="mw-redirect" href="/wiki/Broker_(service-oriented_architecture)"

In [79]:
page_links = get_page_links(page_data)

for k, v in page_links:
    print(k,v)

(u'Free_and_open-source_software', u'free and open-source')
(u'Web_framework', u'web framework')
(u'Python_(programming_language)', u'Python')
(u'Architectural_pattern_(computer_science)', u'architectural pattern')
(u'Django_Software_Foundation', u'Django Software Foundation')
(u'501(c)(3)', u'501(c)(3)')
(u'Reusability', u'reusability')
(u'Don%27t_repeat_yourself', u"don't repeat yourself")
(u'Create,_read,_update_and_delete', u'create, read, update and delete')
(u'Introspection_(computer_science)', u'introspection')
(u'Public_Broadcasting_Service', u'Public Broadcasting Service')
(u'Pinterest', u'Pinterest')
(u'Instagram', u'Instagram')
(u'Mozilla_Foundation', u'Mozilla')
(u'The_Washington_Times', u'The Washington Times')
(u'Disqus', u'Disqus')
(u'Bitbucket', u'Bitbucket')
(u'Nextdoor', u'Nextdoor')
(u'Web_programmer', u'web programmers')
(u'Lawrence_Journal-World', u'Lawrence Journal-World')
(u'Adrian_Holovaty', u'Adrian Holovaty')
(u'Simon_Willison', u'Simon Willison')
(u'BSD_licen

In [9]:
links_score = get_page_links_score(page_links, page_data['full'].get_text())

In [10]:
total_links = 0
for k, v in links_score.items():
    total_links += v

for k, v in sorted(links_score.items(), key=lambda a:a[1], reverse=True):
    print(k,page_links[k],float(v),float(v)/total_links)

(u'django_(disambiguation)', set([u'django']), 136.0, 0.28936170212765955)
(u'python_(programming_language)', set([u'python']), 22.0, 0.04680851063829787)
(u'international_standard_book_number', set([u'isbn']), 8.0, 0.01702127659574468)
(u'ruby_(programming_language)', set([u'ruby']), 4.0, 0.00851063829787234)
(u'model-view-controller', set([u'mvc']), 4.0, 0.00851063829787234)
(u'jython', set([u'jython']), 4.0, 0.00851063829787234)
(u'd_(programming_language)', set([u'd']), 4.0, 0.00851063829787234)
(u'asp.net_mvc', set([u'mvc']), 4.0, 0.00851063829787234)
(u'web_framework', set([u'web frameworks', u'web framework']), 3.0, 0.006382978723404255)
(u'asp.net_core', set([u'core']), 3.0, 0.006382978723404255)
(u'http', set([u'http']), 3.0, 0.006382978723404255)
(u'javascript', set([u'javascript']), 3.0, 0.006382978723404255)
(u'google_app_engine', set([u'google app engine']), 3.0, 0.006382978723404255)
(u'firebird_(database_server)', set([u'firebird']), 3.0, 0.006382978723404255)
(u'django_

In [138]:
#page_data = get_links_score()
#for d in page_data.iteritems():
    #print(d)

In [238]:
#print(page_data['full'].get_text())